In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.stats import norm

##### Initial State: $x_0$

In [6]:
x = np.matrix([[0.0, 0.0, 0.0, 0.0]]).T
print(x, x.shape)

(matrix([[0.],
        [0.],
        [0.],
        [0.]]), (4, 1))


#### Initial Uncertainty $P_0$

In [7]:
P = np.diag([1000.0, 1000.0, 1000.0, 1000.0])
print(P, P.shape)

(array([[1000.,    0.,    0.,    0.],
       [   0., 1000.,    0.,    0.],
       [   0.,    0., 1000.,    0.],
       [   0.,    0.,    0., 1000.]]), (4, 4))


#### Dynamic Matrix $A$

In [8]:
dt = 0.1 # Time Step between Filter Steps

A = np.matrix([[1.0, 0.0, dt, 0.0],
              [0.0, 1.0, 0.0, dt],
              [0.0, 0.0, 1.0, 0.0],
              [0.0, 0.0, 0.0, 1.0]])
print(A, A.shape)

(matrix([[1. , 0. , 0.1, 0. ],
        [0. , 1. , 0. , 0.1],
        [0. , 0. , 1. , 0. ],
        [0. , 0. , 0. , 1. ]]), (4, 4))


#### Measurement Matrix $H$

In [9]:
H = np.matrix([[0.0, 0.0, 1.0, 0.0],
              [0.0, 0.0, 0.0, 1.0]])
print(H, H.shape)

(matrix([[0., 0., 1., 0.],
        [0., 0., 0., 1.]]), (2, 4))


#### Measurement Noise Covariance $R$

In [10]:
ra = 10.0**2

R = np.matrix([[ra, 0.0],
              [0.0, ra]])
print(R, R.shape)

(matrix([[100.,   0.],
        [  0., 100.]]), (2, 2))


#### Process Noise Covariance $Q$

In [11]:
sv = 8.8

G = np.matrix([[0.5*dt**2],
               [0.5*dt**2],
               [dt],
               [dt]])

Q = G*G.T*sv**2

In [12]:
from sympy import Symbol, Matrix
from sympy.interactive import printing
printing.init_printing()
dts = Symbol('dt')
Qs = Matrix([[0.5*dts**2],[0.5*dts**2],[dts],[dts]])
Qs*Qs.T

⎡       4         4        3        3⎤
⎢0.25⋅dt   0.25⋅dt   0.5⋅dt   0.5⋅dt ⎥
⎢                                    ⎥
⎢       4         4        3        3⎥
⎢0.25⋅dt   0.25⋅dt   0.5⋅dt   0.5⋅dt ⎥
⎢                                    ⎥
⎢      3         3       2        2  ⎥
⎢0.5⋅dt    0.5⋅dt      dt       dt   ⎥
⎢                                    ⎥
⎢      3         3       2        2  ⎥
⎣0.5⋅dt    0.5⋅dt      dt       dt   ⎦

#### Identity Matrix $I$

In [13]:
I = np.eye(4)
print(I, I.shape)

(array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]]), (4, 4))


In [21]:
# Preallocation for Plotting
xt = []
yt = []
dxt= []
dyt= []
Zx = []
Zy = []
Px = []
Py = []
Pdx= []
Pdy= []
Rdx= []
Rdy= []
Kx = []
Ky = []
Kdx= []
Kdy= []

def savestates(x, Z, P, R, K):
    xt.append(float(x[0]))
    yt.append(float(x[1]))
    dxt.append(float(x[2]))
    dyt.append(float(x[3]))
    Zx.append(float(Z[0]))
    Zy.append(float(Z[1]))
    Px.append(float(P[0,0]))
    Py.append(float(P[1,1]))
    Pdx.append(float(P[2,2]))
    Pdy.append(float(P[3,3]))
    Rdx.append(float(R[0,0]))
    Rdy.append(float(R[1,1]))
    Kx.append(float(K[0,0]))
    Ky.append(float(K[1,0]))
    Kdx.append(float(K[2,0]))
    Kdy.append(float(K[3,0]))

[[1.74411302]]


#### Measurements

In [14]:

m = 200 # Measurements
vx= 20 # in X
vy= 10 # in Y

mx = np.array(vx+np.random.randn(m))
my = np.array(vy+np.random.randn(m))

measurements = np.vstack((mx,my))

print(measurements.shape)

print('Standard Deviation of Acceleration Measurements=%.2f' % np.std(mx))
print('You assumed %.2f in R.' % R[0,0])

(2, 200)
Standard Deviation of Acceleration Measurements=0.93
You assumed 100.00 in R.


In [25]:
for n in range(len(measurements[0])):
    # Time Update (Prediction)
    # ========================
    # Project the state ahead
    x = A*x
    
    # Project the error covariance ahead
    P = A*P*A.T + Q
    
    
    # Measurement Update (Correction)
    # ===============================
    # Compute the Kalman Gain
    S = H*P*H.T + R
    K = (P*H.T) * np.linalg.pinv(S)

    
    # Update the estimate via z
    Z = measurements[:,n].reshape(2,1)
    y = Z - (H*x)                            # Innovation or Residual
    x = x + (K*y)
    
    # Update the error covariance
    P = (I - (K*H))*P
    
    
    
    # Save states (for Plotting)
    savestates(x, Z, P, R, K)